<a href="https://colab.research.google.com/github/abubakar-zohaib1415/human12/blob/main/Human_In_The_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%pip install -q -U langgraph langchain_google_genai

In [79]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# Setup API key for Google Generative AI
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GEMINI_API_KEY
)

# Define trimmer
trimmer = trim_messages(strategy="last", max_tokens=5, token_counter=len)

# Define the workflow
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    system_prompt = (
        "You Are A Helpfull Assistant for Clothe E-commerce Customer Support Agent. "
        "You Need To Answer Like You Are A ShopKeeper Of Clothe Shop, how clothe shop act, just act like it. "
        "For example: if I say 'Hi, Hello, My name is [Name]', answer greetings. If somebody says 'I want a [product] in [size]', say "
        "'Can I Connect You To A Human Assistant?' wait for a response, and if they say 'yes', respond with: "
        "'Connecting ... To Human Assistant', and then answer like a normal shopkeeper."
    )
    messages = [SystemMessage(content=system_prompt)] + trimmed_messages
    response = llm.invoke(messages)
    return {"messages": response}

# Add node and edge to workflow
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

# Add in-memory checkpointing
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Chat interaction loop
print("Welcome to Clothes Customer Support. Type 'exit' to end the chat.")

messages = []
while True:
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Assistant: Goodbye!")
        break

    # Add user's message to history
    messages.append(HumanMessage(content=user_input))

    # Invoke the app
    response = app.invoke({"messages": messages}, config={"configurable": {"thread_id": "chat-session"}})

    # Get AI response
    ai_response = response["messages"][-1].content
    print(f"Assistant: {ai_response}")

    # Add AI's response to the history
    messages.append(AIMessage(content=ai_response))


Welcome to Clothes Customer Support. Type 'exit' to end the chat.
You: hello
Assistant: Well hello there!  What can I get for you today?

You: exit
Assistant: Goodbye!
